In [1]:
import os
import joblib
import numpy as np
import pandas as pd
import polars as pl

import pandas.api.types
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

import lightgbm as lgb
import xgboost as xgb
import catboost as cb

import lifelines
from lifelines.utils import concordance_index

import sys

print(sys.version)
for i in [np, pd, pl, mpl, sns, lifelines, sklearn, lgb, xgb, cb]:
    try:
        print(i.__name__, i.__version__)
    except:
        print(i.__name__)

3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]
numpy 1.26.4
pandas 2.2.3
polars 1.9.0
matplotlib 3.7.5
seaborn 0.12.2
lifelines 0.30.0
sklearn 1.2.2
lightgbm 4.5.0
xgboost 2.0.3
catboost 1.2.7


In [2]:
import dproc, sgutil, sgpp, sgml, custpp
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate, KFold, ShuffleSplit, train_test_split
from sklearn.impute import SimpleImputer
from lifelines import NelsonAalenFitter

In [3]:
data_path = 'data'
model_path = 'model'

p2 =joblib.load(os.path.join(model_path, 'p2.joblib'))
df_train = p2.transform([os.path.join(data_path, 'train.csv')])

In [5]:
X_bool = ['graft_type', 'prod_type']
X_tri = [
    'arrhythmia', 'cardiac', 'diabetes', 'hepatic_mild', 'hepatic_severe',
    'in_vivo_tcd', 'melphalan_dose', 'mrd_hct', 'obesity', 'peptic_ulcer',
    'prior_tumor', 'psych_disturb', 'pulm_moderate', 'pulm_severe', 'renal_issue',
    'rheum_issue', 'rituximab', 'vent_hist'
]
X_nom = [
    'cmv_status', 'conditioning_intensity', 'cyto_score', 'cyto_score_detail', 'donor_related',
    'dri_score', 'ethnicity', 'gvhd_proph', 'prim_disease_hct', 'race_group', 'sex_match',
    'tbi_status', 'tce_div_match', 'tce_imm_match', 'tce_match'
]
X_na = [
    'arrhythmia_na', 'cardiac_na', 'diabetes_na', 'hepatic_mild_na', 'hepatic_severe_na',
    'obesity_na', 'peptic_ulcer_na', 'prior_tumor_na', 'psych_disturb_na', 'pulm_moderate_na',
    'pulm_severe_na', 'renal_issue_na', 'rheum_issue_na'
]
X_cont = ['age_at_hct', 'donor_age']
X_int = [
    'comorbidity_score', 'hla_high_res_10', 'hla_high_res_6', 'hla_high_res_8', 'hla_low_res_10',
    'hla_low_res_6', 'hla_low_res_8', 'hla_match_a_high', 'hla_match_a_low', 'hla_match_b_high',
    'hla_match_b_low', 'hla_match_drb1_low', 'hla_match_c_high', 'hla_match_c_low', 'hla_match_dqb1_high', 'hla_match_dqb1_low',
    'hla_match_drb1_high', 'hla_nmdp_6', 'karnofsky_score', 'year_hct'
]
X_all = X_tri + X_cont + X_int + X_na + X_bool + X_nom

In [4]:
import torch
from torch.utils.data import TensorDataset

In [25]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler

ct = ColumnTransformer([
    ('std', StandardScaler(), X_cont + X_int[:-1]),
    ('mm', MinMaxScaler(), X_int[-1:]), 
    ('pt', 'passthrough', X_bool + X_tri + X_na)
])
ct.fit_transform(df_train[X_all])

array([[-1.3581534624099731, nan, -0.8535500764846802, ..., False, False,
        False],
       [0.23841628432273865, 1.9525351524353027, 0.65065598487854, ...,
        False, False, False],
       [-0.2206512987613678, nan, -0.8535500764846802, ..., False, False,
        False],
       ...,
       [0.5836153626441956, -1.3038091659545898, 1.6534600257873535, ...,
        True, True, True],
       [-1.8262057304382324, 1.0204088687896729, -0.35214802622795105,
        ..., True, True, False],
       [-1.7793437242507935, -0.7829304933547974, 0.1492539644241333,
        ..., False, False, False]], dtype=object)

In [21]:
df_train[X_cont]

df_train[X_nom].apply(lambda x: x.cat.codes)

,cmv_status,conditioning_intensity,cyto_score,cyto_score_detail,donor_related,dri_score,ethnicity,gvhd_proph,prim_disease_hct,race_group,sex_match,tbi_status,tce_div_match,tce_imm_match,tce_match
ID,,,,,,,,,,,,,,,
0,1,0,0,0,2,4,1,6,0,0,3,0,0,0,0
1,1,1,2,1,1,1,1,7,1,1,4,1,1,1,1
2,1,0,0,0,1,4,1,2,2,0,2,0,1,1,0
3,1,1,2,1,2,3,1,1,3,2,1,0,1,1,1
4,1,1,0,0,1,3,2,9,4,3,3,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28795,4,1,3,1,0,8,1,3,3,0,3,0,4,1,3
28796,2,2,1,4,1,3,2,4,1,4,3,0,2,2,0
28797,2,1,1,2,2,5,1,1,9,4,3,0,2,2,2
